In [4]:
import qiskit 
from mitiq.pec.representations.depolarizing import represent_operation_with_global_depolarizing_noise

qreg = qiskit.QuantumRegister(2, name="test_name")
cnot_circuit = qiskit.QuantumCircuit(qreg)
cnot_circuit.cnot(0, 1)

rep = represent_operation_with_global_depolarizing_noise(cnot_circuit, 0.1)
print(rep)

test_name_0: ───@───
                │
test_name_1: ───X─── = 

1.112
q_0: ───@───
        │
q_1: ───X───

-0.007
q_0: ───@───X───
        │
q_1: ───X───────

-0.007
q_0: ───@───Y───
        │
q_1: ───X───────

-0.007
q_0: ───@───Z───
        │
q_1: ───X───────

-0.007
q_0: ───@───────
        │
q_1: ───X───X───

-0.007
q_0: ───@───────
        │
q_1: ───X───Y───

-0.007
q_0: ───@───────
        │
q_1: ───X───Z───

-0.007
q_0: ───@───X───
        │
q_1: ───X───X───

-0.007
q_0: ───@───X───
        │
q_1: ───X───Y───

-0.007
q_0: ───@───X───
        │
q_1: ───X───Z───

-0.007
q_0: ───@───Y───
        │
q_1: ───X───X───

-0.007
q_0: ───@───Y───
        │
q_1: ───X───Y───

-0.007
q_0: ───@───Y───
        │
q_1: ───X───Z───

-0.007
q_0: ───@───Z───
        │
q_1: ───X───X───

-0.007
q_0: ───@───Z───
        │
q_1: ───X───Y───

-0.007
q_0: ───@───Z───
        │
q_1: ───X───Z───


In [5]:
from mitiq.interface import convert_to_mitiq, convert_from_mitiq
converted, in_type = convert_to_mitiq(cnot_circuit)
print(converted)

test_name_0: ───@───
                │
test_name_1: ───X───


In [6]:
from typing import List
from cirq import Operation, X, Y, Z, Circuit
post_ops: List[List[Operation]]
qubits = converted.all_qubits()
print(qubits)

frozenset({cirq.NamedQubit('test_name_0'), cirq.NamedQubit('test_name_1')})


In [7]:
from cirq import X, Y, Z, Circuit
noise_level = 0.1
# The single-qubit case: linear combination of 1Q Paulis
if len(qubits) == 1:
    q = tuple(qubits)[0]

    epsilon = 4 / 3 * noise_level
    alpha_pos = 1 + ((3 / 4) * epsilon / (1 - epsilon))
    alpha_neg = -(1 / 4) * epsilon / (1 - epsilon)

    alphas = [alpha_pos] + 3 * [alpha_neg]
    post_ops = [[]]  # for alpha_pos, we do nothing, rather than I
    post_ops += [[P(q)] for P in [X, Y, Z]]  # 1Q Paulis

# The two-qubit case: linear combination of 2Q Paulis
elif len(qubits) == 2:
    q0, q1 = qubits

    epsilon = 16 / 15 * noise_level
    alpha_pos = 1 + ((15 / 16) * epsilon / (1 - epsilon))
    alpha_neg = -(1 / 16) * epsilon / (1 - epsilon)

    alphas = [alpha_pos] + 15 * [alpha_neg]
    post_ops = [[]]  # for alpha_pos, we do nothing, rather than I x I
    post_ops += [[P(q0)] for P in [X, Y, Z]]  # 1Q Paulis for q0
    post_ops += [[P(q1)] for P in [X, Y, Z]]  # 1Q Paulis for q1
    post_ops += [
        [Pi(q0), Pj(q1)] for Pi in [X, Y, Z] for Pj in [X, Y, Z]
    ]  # 2Q Paulis

# Basis of implementable operations as circuits.
imp_op_circuits = [converted + Circuit(op) for op in post_ops]
print(imp_op_circuits)

[cirq.Circuit([
    cirq.Moment(
        cirq.CNOT(cirq.NamedQubit('test_name_0'), cirq.NamedQubit('test_name_1')),
    ),
]), cirq.Circuit([
    cirq.Moment(
        cirq.CNOT(cirq.NamedQubit('test_name_0'), cirq.NamedQubit('test_name_1')),
    ),
    cirq.Moment(
        cirq.X(cirq.NamedQubit('test_name_0')),
    ),
]), cirq.Circuit([
    cirq.Moment(
        cirq.CNOT(cirq.NamedQubit('test_name_0'), cirq.NamedQubit('test_name_1')),
    ),
    cirq.Moment(
        cirq.Y(cirq.NamedQubit('test_name_0')),
    ),
]), cirq.Circuit([
    cirq.Moment(
        cirq.CNOT(cirq.NamedQubit('test_name_0'), cirq.NamedQubit('test_name_1')),
    ),
    cirq.Moment(
        cirq.Z(cirq.NamedQubit('test_name_0')),
    ),
]), cirq.Circuit([
    cirq.Moment(
        cirq.CNOT(cirq.NamedQubit('test_name_0'), cirq.NamedQubit('test_name_1')),
    ),
    cirq.Moment(
        cirq.X(cirq.NamedQubit('test_name_1')),
    ),
]), cirq.Circuit([
    cirq.Moment(
        cirq.CNOT(cirq.NamedQubit('test_name_0'),

In [8]:
from mitiq.pec.types import NoisyOperation
# Convert back to input type.
imp_op_circuits_converted = [convert_from_mitiq(c, in_type) for c in imp_op_circuits]
print(imp_op_circuits_converted)


[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac250ed30>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac2a18430>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac277cd30>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac2758340>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac2367c10>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac2136400>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac21fe0a0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac269aeb0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac27b1fd0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac26a6340>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac2242460>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac2b2a5e0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3ac28746d0>, <qiskit.circuit.quantumcircuit.QuantumCircuit obje